## Using Keras Sequential Model and Functional API

In [1]:
import os, random, numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

%matplotlib inline

np.set_printoptions(precision=2, suppress=True)

In [2]:
def softmax(logits):
    exp_logits = np.exp(logits)
    return exp_logits / np.sum(exp_logits)

logits = np.array([2.0, 1.0, 0.1])
probabilities = softmax(logits)
print(probabilities, sum(probabilities))

[0.66 0.24 0.1 ] 1.0


In [3]:
# Load MNIST data

seed_value = 1234

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


# Sequential Model

In [4]:
# Sequential Model (Very convenient, not very flexible)

random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

model_1 = keras.Sequential(
    [
        keras.Input(shape=(x_train.shape[-1],)),
        layers.Dense(512, activation="relu"),
        layers.Dense(256, activation="relu"),
        layers.Dense(10),
    ]
)

model_1.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

model_1.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
print("\nloss = {:.4f}, accuracy = {:.4f}". format(*model_1.evaluate(x_test, y_test, batch_size=32, verbose=2)))

Epoch 1/5
1875/1875 - 7s - 4ms/step - accuracy: 0.9435 - loss: 0.1852
Epoch 2/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9765 - loss: 0.0768
Epoch 3/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9848 - loss: 0.0485
Epoch 4/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9879 - loss: 0.0374
Epoch 5/5
1875/1875 - 7s - 4ms/step - accuracy: 0.9899 - loss: 0.0302
313/313 - 1s - 2ms/step - accuracy: 0.9802 - loss: 0.0780

loss = 0.0780, accuracy = 0.9802


In [5]:
# Another way to use Sequential Model

random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)

model_1b = keras.Sequential()
model_1b.add(keras.Input(shape=(x_train.shape[-1],)))
model_1b.add(layers.Dense(512, activation="relu", name="fc_layer1"))
model_1b.add(layers.Dense(256, activation="relu", name="fc_layer2"))
model_1b.add(layers.Dense(10, name="outputs"))

model_1b.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

model_1b.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ fc_layer1 (Dense)               │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc_layer2 (Dense)               │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ outputs (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 535,818 (2.04 MB)

 Trainable params: 535,818 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model_1b.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
print("\nloss = {:.4f}, accuracy = {:.4f}". format(*model_1b.evaluate(x_test, y_test, batch_size=32, verbose=2)))

Epoch 1/5
1875/1875 - 7s - 4ms/step - accuracy: 0.9435 - loss: 0.1852
Epoch 2/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9765 - loss: 0.0768
Epoch 3/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9848 - loss: 0.0485
Epoch 4/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9879 - loss: 0.0374
Epoch 5/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9899 - loss: 0.0302
313/313 - 1s - 2ms/step - accuracy: 0.9802 - loss: 0.0780

loss = 0.0780, accuracy = 0.9802


## Categorical Crossentropy

In [7]:
from keras.utils import to_categorical

y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)

model_1c = keras.Sequential()
model_1c.add(keras.Input(shape=(x_train.shape[-1],)))
model_1c.add(layers.Dense(512, activation="relu", name="fc_layer1"))
model_1c.add(layers.Dense(256, activation="relu", name="fc_layer2"))
model_1c.add(layers.Dense(10, activation="softmax", name="outputs"))

model_1c.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

model_1c.summary()

model_1c.fit(x_train, y_train_one_hot, batch_size=32, epochs=5, verbose=2)
print("\nloss = {:.4f}, accuracy = {:.4f}". format(*model_1c.evaluate(x_test, y_test_one_hot, batch_size=32, verbose=2)))

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ fc_layer1 (Dense)               │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc_layer2 (Dense)               │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ outputs (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 535,818 (2.04 MB)

 Trainable params: 535,818 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
1875/1875 - 7s - 4ms/step - accuracy: 0.9435 - loss: 0.1852
Epoch 2/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9765 - loss: 0.0768
Epoch 3/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9848 - loss: 0.0485
Epoch 4/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9879 - loss: 0.0374
Epoch 5/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9899 - loss: 0.0302
313/313 - 1s - 2ms/step - accuracy: 0.9802 - loss: 0.0780

loss = 0.0780, accuracy = 0.9802


In [8]:
y_pred = model_1c.predict(x_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [9]:
y_pred.argmax(axis=1)

array([7, 2, 1, ..., 4, 5, 6])

In [10]:
y_test

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [11]:
print((y_test == y_pred.argmax(axis=1)).mean())

0.9802


# Functional API

In [12]:
# Functional API (A bit more flexible)

random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)

inputs = keras.Input(shape=(x_train.shape[-1],), name="inputs")
x = layers.Dense(512, activation="relu", name="fc_layer1")(inputs)
x = layers.Dense(256, activation="relu", name="fc_layer2")(x)
outputs = layers.Dense(10, activation="softmax", name="outputs")(x)

model_2 = keras.Model(inputs=inputs, outputs=outputs)

model_2.summary()

model_2.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

model_2.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
print("\nloss = {:.4f}, accuracy = {:.4f}". format(*model_2.evaluate(x_test, y_test, batch_size=32, verbose=2)))

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inputs (InputLayer)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc_layer1 (Dense)               │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc_layer2 (Dense)               │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ outputs (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 535,818 (2.04 MB)

 Trainable params: 535,818 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9435 - loss: 0.1852
Epoch 2/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9765 - loss: 0.0768
Epoch 3/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9848 - loss: 0.0485
Epoch 4/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9879 - loss: 0.0374
Epoch 5/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9899 - loss: 0.0302
313/313 - 1s - 2ms/step - accuracy: 0.9802 - loss: 0.0780

loss = 0.0780, accuracy = 0.9802


# More on Functional API

In [13]:
# Functional API (A bit more flexible)

random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)

inputs = keras.Input(shape=(x_train.shape[-1],), name="inputs")
x1 = layers.Dense(512, activation="relu", name="fc_layer1")(inputs)
x2 = layers.Dense(256, activation="relu", name="fc_layer2")(x1)
x = layers.Concatenate(axis=1, name="concat_layer1")([x1, x2])
x = layers.Dense(64, activation="relu", name="fc_layer3")(x)
outputs = layers.Dense(10, activation="softmax", name="outputs")(x)

model_3 = keras.Model(inputs=inputs, outputs=outputs)

model_3.summary()

model_3.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

model_3.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
print("\nloss = {:.4f}, accuracy = {:.4f}". format(*model_3.evaluate(x_test, y_test, batch_size=32, verbose=2)))

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer) │ (None, 784)       │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fc_layer1 (Dense)   │ (None, 512)       │    401,920 │ inputs[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fc_layer2 (Dense)   │ (None, 256)       │    131,328 │ fc_layer1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concat_layer1       │ (None, 768)       │          0 │ fc_layer1[0][0],  │
│ (Concatenate)       │                   │            │ fc_layer2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fc_layer3 (Dense)   │ (None, 64)        │     49,216 │ concat_layer1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ outputs (Dense)     │ (None, 10)        │        650 │ fc_layer3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 583,114 (2.22 MB)

 Trainable params: 583,114 (2.22 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
1875/1875 - 7s - 4ms/step - accuracy: 0.9422 - loss: 0.1910
Epoch 2/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9749 - loss: 0.0810
Epoch 3/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9825 - loss: 0.0538
Epoch 4/5
1875/1875 - 7s - 4ms/step - accuracy: 0.9868 - loss: 0.0402
Epoch 5/5
1875/1875 - 7s - 4ms/step - accuracy: 0.9890 - loss: 0.0340
313/313 - 1s - 2ms/step - accuracy: 0.9768 - loss: 0.0967

loss = 0.0967, accuracy = 0.9768
